In [1]:

import pandas as pd
import re
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import math
train_data = pd.read_csv('data/train.tsv', sep='\t',names=['text', 'emotion', 'id'])
dev_data = pd.read_csv('data/dev.tsv',sep='\t',names=['text', 'emotion', 'id'])
#train_data = train_data.iloc[:5000]
#dev_data = dev_data.iloc[:500]
test_data = pd.read_csv('data/test.tsv', sep='\t',names= ['text','emotion','id'])
#test_data = test_data.iloc[:500]
print(train_data.head())

                                                text emotion       id
0  My favourite food is anything I didn't have to...      27  eebbqej
1  Now if he does off himself, everyone will thin...      27  ed00q6i
2                     WHY THE FUCK IS BAYLESS ISOING       2  eezlygj
3                        To make her feel threatened      14  ed7ypvh
4                             Dirty Southern Wankers       3  ed0bdzj


In [2]:
class WordDict:
    def __init__(self, words):
        # Vérifie que 'words' est un ensemble (set)
        assert type(words) == set
        # Crée un mapping mot -> entier
        self.word_to_idx = {word: idx for idx, word in enumerate(sorted(words))}
        # Crée un mapping entier -> mot
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}
    
    def word_to_id(self, word):
        assert type(word) == str
        return self.word_to_idx[word]

    def id_to_word(self, idx):
        assert type(idx) == int
        return self.idx_to_word[idx]

    def __len__(self):
        return len(self.word_to_idx)

In [3]:
train_words = set()
for sentence in train_data['text']:  # 'train_data' contient les phrases sous forme de texte brut
    clean_sentence = re.sub(r"[^\w\s]", "", sentence).lower()
    train_words.update(clean_sentence.split())  # Ajoute chaque mot de la phrase
# Ajouter les tokens spéciaux
train_words.update(["<bos>", "<eos>","<UNK>","<PAD>"])

# Créer une instance de WordDict
word_dict = WordDict(train_words)

# Vérifier la taille du dictionnaire
print("Nombre de mots uniques :", len(word_dict))


Nombre de mots uniques : 28277


In [4]:

def tokenize_sentences(sentences, word_dict):
    """
    Tokenize une liste de phrases en utilisant un dictionnaire de mots.
    Si un mot n'est pas dans le dictionnaire, il est remplacé par le token <UNK>.
    
    Args:
        sentences (list of str): Liste de phrases à tokenizer.
        word_dict (WordDict): Instance de WordDict contenant le vocabulaire.

    Returns:
        list of list of int: Liste de phrases tokenisées sous forme d'entiers.
    """
    # Récupérer l'ID du token <UNK>
    unk_id = word_dict.word_to_id("<UNK>")
    tokenized_sentences = []

    for sentence in sentences:
        # Nettoyer la phrase : supprimer la ponctuation et mettre en minuscule
        clean_sentence = re.sub(r"[^\w\s]", "", sentence).lower()
        # Découper en mots
        words = clean_sentence.split()
        # Convertir chaque mot en ID, ou utiliser <UNK> si le mot n'est pas dans le dictionnaire
        tokenized_sentence = [word_dict.word_to_id(word) if word in word_dict.word_to_idx else unk_id for word in words]
        # Ajouter la phrase tokenisée à la liste
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences


In [5]:
print(tokenize_sentences(['Hello how are you?',"I'm fine and you ?"],word_dict))

[[11767, 12222, 2033, 28143], [12536, 9662, 1699, 28143]]


In [6]:
train_data['text'].tolist()

["My favourite food is anything I didn't have to cook myself.",
 'Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead',
 'WHY THE FUCK IS BAYLESS ISOING',
 'To make her feel threatened',
 'Dirty Southern Wankers',
 "OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe PlAyOfFs! Dumbass Broncos fans circa December 2015.",
 'Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝',
 'We need more boards and to create a bit more space for [NAME]. Then we’ll be good.',
 'Damn youtube and outrage drama is super lucrative for reddit',
 'It might be linked to the trust factor of your friend.',
 'Demographics? I don’t know anybody under 35 who has cable tv.',
 "Aww... she'll probably come around eventually, I'm sure she was just jealous of [NAME]... I mean, what woman wouldn't be! lol ",
 'Hello everyone. Im from Toronto as well. Can call and visit in personal if needed.',
 "R/s

In [7]:
def generate_ngrams_with_padding(tokenized_sentence, n, word_dict):
    """
    Génère des n-grams pour une phrase tokenisée en ajoutant du padding <BOS> au début 
    et <EOS> à la fin.
    
    Args:
        tokenized_sentence (list of str): Liste de tokens représentant une phrase.
        n (int): Taille des n-grams.
        word_dict (WordDict): Dictionnaire des mots avec leurs ID.
    
    Returns:
        list of tuple: Liste de n-grams sous la forme ([mot1, mot2, ..., mot_{n-1}], mot_n).
    """
    # Récupérer l'ID des tokens spéciaux
    bos_id = word_dict.word_to_id("<bos>")
    eos_id = word_dict.word_to_id("<eos>")
    
    # Ajouter des tokens <BOS> au début et <EOS> à la fin
    padded_sentence = [bos_id] * (n - 1) + tokenized_sentence + [eos_id]
    
    # Générer les n-grams
    ngrams = []
    for i in range(len(padded_sentence) - n + 1):
        context = padded_sentence[i:i + n - 1]  # Les n-1 premiers mots
        target = padded_sentence[i + n - 1]  # Le n-ème mot
        ngrams.append((context, target))
    
    return ngrams


In [8]:
# Exemple d'une phrase tokenisée
tokenized_sentence = [1, 2, 3, 4]  # Représente "Hello how are you"

# Génération des n-grams
n = 3
ngrams = generate_ngrams_with_padding(tokenized_sentence, n,word_dict)

# Afficher les résultats
print("N-grams générés :", ngrams)


N-grams générés : [([835, 835], 1), ([835, 1], 2), ([1, 2], 3), ([2, 3], 4), ([3, 4], 836)]


In [60]:


class MLPModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim,n_gram):
        """
        Initialise le modèle MLP avec une couche d'Embedding et plusieurs couches linéaires.

        Args:
            vocab_size (int): Taille du vocabulaire (nombre de mots dans le dictionnaire)
            embed_dim (int): Dimension des embeddings (représentation dense des mots)
            n_layers (int): Nombre de couches linéaires
            hidden_dim (int): Dimension des couches cachées
            output_dim (int): Dimension de la sortie  (par exemple, la taille du vocabulaire pour la classification)
        """
        super(MLPModel, self).__init__()
        
        # Couche d'Embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Couches linéaires
        input_dim = embed_dim * (n_gram - 1)  # Le nombre d'entrées est le produit de la taille d'un n-gram
        
        # Création des couches linéaires
        self.L1 = nn.Linear(input_dim, hidden_dim)
        self.L2 = nn.Linear(hidden_dim, hidden_dim)
        self.L3 = nn.Linear(hidden_dim,hidden_dim)
        self.Lnout = nn.Linear(hidden_dim, output_dim)
        
        #self.softmax = nn.Softmax(dim=-1)
        self.Activation = nn.ReLU()

    def forward(self, x):
        """
        Effectue une passe avant dans le modèle MLP.

        Args:
            x (Tensor): Entrée, taille (batch_size, n-1)
        
        Returns:
            Tensor: La sortie du modèle, taille (batch_size, output_dim)
        """
        # Appliquer l'Embedding
        embedded = self.embedding(x)  # Taille (batch_size, n-1, embed_dim)
        
        # Aplatir l'entrée pour correspondre à la taille de l'entrée des couches linéaires
        embedded = embedded.view(embedded.size(0), -1)  # Aplatir en (batch_size, n-1*embed_dim)
        
        # Appliquer la première couche linéaire et ReLU
        hidden1 = self.L1(embedded)
        hidden1 = torch.relu(hidden1)
        
        # Appliquer la deuxième couche linéaire et ReLU
        #hidden2 = self.L2(hidden1)
        #hidden2 = torch.relu(hidden2)
        #hidden3 = self.L3(hidden2)
        #hidden3 = torch.relu(hidden2)
        # Appliquer la couche de sortie
        output = self.Lnout(hidden1)
        
        #output = self.Activation(output)
        
        return output



In [33]:
# Fonction d'entraînement mise à jour avec les n-grams
def train(model, train_data, word_dict, ngram_size, batch_size, epochs, device,dev_data):
    """
    Entraîne un modèle MLP pour prédire le mot suivant dans une séquence de n-grams.

    Args:
    - model: Le modèle MLP.
    - train_data: Liste de phrases à tokenizer et utiliser pour l'entraînement.
    - word_dict: Dictionnaire des mots.
    - ngram_size: Taille des n-grams.
    - batch_size: Taille du batch.
    - epochs: Nombre d'époques pour l'entraînement.
    - device: Le périphérique (CPU ou GPU).
    """
    
    # Tokenisation des phrases
    tokenized_sentences = tokenize_sentences(train_data['text'].tolist(), word_dict)
    
    # Générer les n-grams pour l'entraînement
    input_sequences = []
    target_words = []
    
    for sentence in tokenized_sentences:
        ngrams = generate_ngrams_with_padding(sentence, ngram_size, word_dict)
        for context, target in ngrams:
            input_sequences.append(context)
            target_words.append(target)
    
    # Convertir les listes en tensors PyTorch
    input_tensor = torch.tensor(input_sequences, dtype=torch.long)
    target_tensor = torch.tensor(target_words, dtype=torch.long)

    # Créer un DataLoader pour itérer sur les batches
    dataset = TensorDataset(input_tensor, target_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Définir la fonction de perte et l'optimiseur
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Entraînement
    model.to(device)  # Déplacer le modèle sur le périphérique (GPU ou CPU)
    model.train()     # Passer en mode entraînement
    
    for epoch in range(epochs):
        epoch_loss = 0
        for inputs, targets in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Passer les inputs dans le modèle
            optimizer.zero_grad()  # Réinitialiser les gradients
            outputs = model(inputs)  # Obtenir les prédictions du modèle
            
            # Calculer la perte
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()
            
            # Calculer les gradients et mettre à jour les poids
            loss.backward()
            optimizer.step()
        model.eval()  # Passer en mode évaluation
        with torch.no_grad():  # Ne pas calculer les gradients pendant la validation
        # Tokenisation des phrases de validation
            tokenized_dev_sentences = tokenize_sentences(dev_data['text'].tolist(), word_dict)
            dev_input_sequences = []
            dev_target_words = []
                    
            # Générer les n-grams pour la validation
            for sentence in tokenized_dev_sentences:
                ngrams = generate_ngrams_with_padding(sentence, ngram_size, word_dict)
                for context, target in ngrams:
                    dev_input_sequences.append(context)
                    dev_target_words.append(target)
                    
                    # Convertir les listes en tensors PyTorch
            dev_input_tensor = torch.tensor(dev_input_sequences, dtype=torch.long)
            dev_target_tensor = torch.tensor(dev_target_words, dtype=torch.long)
                    
                    # Créer un DataLoader pour les données de validation
            dev_dataset = TensorDataset(dev_input_tensor, dev_target_tensor)
            dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
                    
                    # Calculer la perte sur les données de validation
            dev_loss = 0
            for inputs, targets in dev_dataloader:
                inputs, targets = inputs.to(device), targets.to(device)
                        
                        # Passer les inputs dans le modèle
                outputs = model(inputs)  # Obtenir les prédictions du modèle
                        
                        # Calculer la perte
                loss = criterion(outputs, targets)
                dev_loss += loss.item()
                    
                    # Afficher la perte de validation
            print(f"Validation Loss after Epoch {epoch+1}/{epochs}: {dev_loss / len(dev_dataloader)}")
            # Afficher la perte moyenne pour chaque époque
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")
        
    print("Entraînement terminé!")

In [11]:
class Perplexity:
    def __init__(self):
        """
        Initialise les variables nécessaires pour calculer la perplexité.
        """
        self.log_sum = 0  # Somme cumulée des -log(proba)
        self.word_count = 0  # Nombre total de mots traités

    def reset(self):
        """
        Réinitialise les valeurs pour un nouveau calcul de perplexité.
        """
        self.log_sum = 0
        self.word_count = 0

    def add_sentence(self, log_probs):
        """
        Ajoute les probabilités logarithmiques d'une phrase à la somme.
        
        Args:
        - log_probs: Liste ou tableau des log probabilités des mots dans une phrase.
        """
        self.log_sum += -sum(log_probs)  # On accumule les -log(proba)
        self.word_count += len(log_probs)  # On incrémente le nombre total de mots

    def compute_perplexity(self):
        """
        Calcule la perplexité en fonction des données accumulées.
        
        Returns:
        - perplexity: La perplexité calculée (float).
        """
        if self.word_count == 0:
            raise ValueError("Aucun mot n'a été ajouté. Impossible de calculer la perplexité.")
        
        # Moyenne des -log(proba)
        avg_log_prob = self.log_sum / self.word_count
        
        # Calcul de la perplexité
        perplexity = math.exp(avg_log_prob)
        return (perplexity,self.word_count)

In [12]:
tokenized_sentences = tokenize_sentences(train_data['text'].tolist(), word_dict)
    
    # Générer les n-grams pour l'entraînement
input_sequences = []
target_words = []
    
for sentence in tokenized_sentences:
    ngrams = generate_ngrams_with_padding(sentence, 4, word_dict)
    for context, target in ngrams:
        input_sequences.append(context)
        target_words.append(target)

In [13]:
input_sequences

[[835, 835, 835],
 [835, 835, 16297],
 [835, 16297, 9393],
 [16297, 9393, 9943],
 [9393, 9943, 13293],
 [9943, 13293, 1880],
 [13293, 1880, 12395],
 [1880, 12395, 7277],
 [12395, 7277, 11612],
 [7277, 11612, 25301],
 [11612, 25301, 5876],
 [25301, 5876, 16300],
 [835, 835, 835],
 [835, 835, 16929],
 [835, 16929, 12479],
 [16929, 12479, 11647],
 [12479, 11647, 7702],
 [11647, 7702, 17151],
 [7702, 17151, 11916],
 [17151, 11916, 8876],
 [11916, 8876, 27601],
 [8876, 27601, 25026],
 [27601, 25026, 11840],
 [25026, 11840, 11618],
 [11840, 11618, 860],
 [11618, 860, 14086],
 [860, 14086, 21834],
 [14086, 21834, 27679],
 [21834, 27679, 18187],
 [27679, 18187, 13025],
 [18187, 13025, 17148],
 [13025, 17148, 1109],
 [17148, 1109, 6682],
 [835, 835, 835],
 [835, 835, 27548],
 [835, 27548, 24932],
 [27548, 24932, 10268],
 [24932, 10268, 13293],
 [10268, 13293, 2850],
 [13293, 2850, 13312],
 [835, 835, 835],
 [835, 835, 25301],
 [835, 25301, 14964],
 [25301, 14964, 11798],
 [14964, 11798, 9456],


In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(word_dict)  # Taille du vocabulaire (dictionnaire de mots)
embed_dim = 16  # Dimension de l'embedding
n_layers = 1  # Nombre de couches linéaires
hidden_dim = 32  # Dimension des couches cachées
output_dim = vocab_size  # La sortie doit avoir la même taille que le vocabulaire (prédiction d'un mot)

model = MLPModel(vocab_size=vocab_size, embed_dim=embed_dim, hidden_dim=hidden_dim, output_dim=output_dim,n_gram=2)

train(model, train_data, word_dict, ngram_size=2, batch_size=128, epochs=3, device=device,dev_data=dev_data)

Epoch 1/3: 100%|██████████| 4669/4669 [03:02<00:00, 25.60it/s]


Validation Loss after Epoch 1/3: 6.315972705886946
Epoch 1/3, Loss: 6.55934572138146


Epoch 2/3: 100%|██████████| 4669/4669 [03:48<00:00, 20.41it/s]


Validation Loss after Epoch 2/3: 6.278362356100705
Epoch 2/3, Loss: 5.996452584514834


Epoch 3/3: 100%|██████████| 4669/4669 [03:52<00:00, 20.07it/s]


Validation Loss after Epoch 3/3: 6.345683877820411
Epoch 3/3, Loss: 5.801496244493045
Entraînement terminé!


In [15]:
def test(model, data_test, word_dict, ngram_size, batch_size, device):
    """
    Évalue le modèle sur les données de test.

    Args:
    - model: Le modèle MLP.
    - data_test: Données de test.
    - word_dict: Dictionnaire des mots.
    - ngram_size: Taille des n-grams.
    - batch_size: Taille du batch.
    - device: Le périphérique (CPU ou GPU).
    """
    
    # Tokenisation des phrases de test
    tokenized_test = tokenize_sentences(data_test['text'].tolist(), word_dict)
    perplexity = Perplexity()
    # Générer les n-grams pour le test
    input_sequences_test, target_words_test = [], []
    for sentence in tokenized_test:
        ngrams = generate_ngrams_with_padding(sentence, ngram_size, word_dict)
        for context, target in ngrams:
            input_sequences_test.append(context)
            target_words_test.append(target)
    
    # Convertir les listes en tensors PyTorch
    input_tensor_test = torch.tensor(input_sequences_test, dtype=torch.long)
    target_tensor_test = torch.tensor(target_words_test, dtype=torch.long)
    
    # Créer un DataLoader pour itérer sur les batches
    test_dataset = TensorDataset(input_tensor_test, target_tensor_test)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Définir la fonction de perte
    criterion = nn.CrossEntropyLoss()
    
    # Évaluation
    model.eval()  # Passer en mode évaluation
    with torch.no_grad():
        test_loss = 0
        correct = 0
        total = 0
        for inputs, targets in  tqdm(test_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            # Calculer la perte
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            log_probs = torch.log_softmax(outputs, dim=1)  # Taille : [batch_size, vocab_size]
            log_probs_targets = log_probs[range(len(targets)), targets] 
            perplexity.add_sentence(log_probs_targets)
            # Calculer la précision
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        test_accuracy = correct / total
        print(f"Test Loss: {test_loss / len(test_dataloader)}, Test Accuracy: {test_accuracy * 100}%, Test Perplexity :{perplexity.compute_perplexity()}")


In [62]:
test(model, test_data, word_dict, ngram_size=2, batch_size=128, device=device)

100%|██████████| 579/579 [00:15<00:00, 37.84it/s]

Test Loss: 6.333929318838169, Test Accuracy: 12.85729709303895%, Test Perplexity :(563.3098769099696, 74098)


In [68]:
#LSTM avec random Dropout

class LSTMWithRandomDropout(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMWithRandomDropout, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.Relu = nn.ReLU()
        self.L1 = nn.Linear(hidden_dim,hidden_dim)
        
    def forward(self, x, dropout_rate=None):
        if dropout_rate is not None:
            self.dropout.p = dropout_rate  
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        dropped_out = self.dropout(lstm_out[:, -1, :])  # Dropout appliqué à la sortie finale
        output = self.fc(dropped_out)
        return output

In [69]:
class LSTMWithVariationalDropout(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout_rate=0.5):
        super(LSTMWithVariationalDropout, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)  # Optionnel pour d'autres parties, comme l'entrée ou la sortie
        self.hidden_dim = hidden_dim

    def forward(self, x):
        embedded = self.embedding(x)
        
        # LSTM avec variational dropout activé par le paramètre dropout
        lstm_out, _ = self.lstm(embedded)
        
        # On applique éventuellement un dropout sur la sortie finale (optionnel, mais souvent utile)
        lstm_out_last = self.dropout(lstm_out[:, -1, :])  # Dernière sortie temporelle
        output = self.fc(lstm_out_last)
        return output

In [41]:
def target_pairs(tokenized_sentences, word_dict, max_context_length):
    """
    Crée des paires (contexte, cible) à partir des phrases tokenisées.
    
    Args:
        tokenized_sentences (list of list of int): Liste de phrases tokenisées.
        word_dict (WordDict): Instance de WordDict contenant les tokens spéciaux.
        max_context_length (int): Longueur maximale du contexte.
    
    Returns:
        list of tuple: Liste de tuples (contexte, cible).
    """
    pairs = []
    for sentence in tokenized_sentences:
        bos_id = word_dict.word_to_id("<bos>")
        sentence = [bos_id] * max_context_length + sentence + [word_dict.word_to_id("<eos>")]
        
        for i in range(max_context_length, len(sentence)):
            context = sentence[max(0, i - max_context_length):i]
            target = sentence[i]
            pairs.append((context, target))
    return pairs

In [18]:
print(target_pairs([[1,2,3,4,5,6,7,8,9,10]],word_dict,3))

[([835, 835, 835], 1), ([835, 835, 1], 2), ([835, 1, 2], 3), ([1, 2, 3], 4), ([2, 3, 4], 5), ([3, 4, 5], 6), ([4, 5, 6], 7), ([5, 6, 7], 8), ([6, 7, 8], 9), ([7, 8, 9], 10), ([8, 9, 10], 836)]


In [48]:
word_dict

In [48]:
def train_lstm(model, train_data, word_dict, n_size, batch_size, epochs, device,dev_data):
    """
    Entraîne un modèle lstm

    Args:
    - model: Le modèle lstm.
    - train_data: Liste de phrases à tokenizer et utiliser pour l'entraînement.
    - word_dict: Dictionnaire des mots.
    - batch_size: Taille du batch.
    - epochs: Nombre d'époques pour l'entraînement.
    - device: Le périphérique (CPU ou GPU).
    """
    
    # Tokenisation des phrases
    tokenized_sentences = tokenize_sentences(train_data['text'].tolist(), word_dict)
    
    # Générer les n-grams pour l'entraînement
    input_sequences = []
    target_words = []
    
    for sentence in tokenized_sentences:
        pairs =  target_pairs([sentence], word_dict,n_size)
        for context, target in pairs:
            input_sequences.append(context)
            target_words.append(target)
    
    # Convertir les listes en tensors PyTorch
    input_tensor = torch.tensor(input_sequences, dtype=torch.long)
    target_tensor = torch.tensor(target_words, dtype=torch.long)

    # Créer un DataLoader pour itérer sur les batches
    dataset = TensorDataset(input_tensor, target_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Définir la fonction de perte et l'optimiseur
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    # Entraînement
    model.to(device)  # Déplacer le modèle sur le périphérique (GPU ou CPU)
    model.train()     # Passer en mode entraînement
    
    for epoch in range(epochs):
        epoch_loss = 0
        for inputs, targets in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Passer les inputs dans le modèle
            optimizer.zero_grad()  # Réinitialiser les gradients
            outputs = model(inputs)  # Obtenir les prédictions du modèle
            
            # Calculer la perte
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()
            
            # Calculer les gradients et mettre à jour les poids
            loss.backward()
            optimizer.step()
        model.eval()  # Passer en mode évaluation
        with torch.no_grad():  # Ne pas calculer les gradients pendant la validation
        # Tokenisation des phrases de validation
            tokenized_dev_sentences = tokenize_sentences(dev_data['text'].tolist(), word_dict)
            dev_input_sequences = []
            dev_target_words = []
                    
            # Générer les pairs pour la validation
            for sentence in tokenized_dev_sentences:
                pairs =  target_pairs([sentence], word_dict,n_size)

                for context, target in pairs:
                    dev_input_sequences.append(context)
                    dev_target_words.append(target)
                    
                    # Convertir les listes en tensors PyTorch
            dev_input_tensor = torch.tensor(dev_input_sequences, dtype=torch.long)
            dev_target_tensor = torch.tensor(dev_target_words, dtype=torch.long)
                    
                    # Créer un DataLoader pour les données de validation
            dev_dataset = TensorDataset(dev_input_tensor, dev_target_tensor)
            dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
                    
                    # Calculer la perte sur les données de validation
            dev_loss = 0
            for inputs, targets in dev_dataloader:
                inputs, targets = inputs.to(device), targets.to(device)
                        
                        # Passer les inputs dans le modèle
                outputs = model(inputs)  # Obtenir les prédictions du modèle
                        
                        # Calculer la perte
                loss = criterion(outputs, targets)
                dev_loss += loss.item()
                    
                    # Afficher la perte de validation
            print(f"Validation Loss after Epoch {epoch+1}/{epochs}: {dev_loss / len(dev_dataloader)}")
            # Afficher la perte moyenne pour chaque époque
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")
        
    print("Entraînement terminé!")

In [49]:
word_dict.word_to_id("<bos>")

835

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(word_dict)  # Taille du vocabulaire (dictionnaire de mots)
embed_dim = 16  # Dimension de l'embedding
hidden_dim = 32  # Dimension des couches cachées
output_dim = vocab_size  # La sortie doit avoir la même taille que le vocabulaire (prédiction d'un mot)
n_size = 5
batch_size = 64
epochs = 3
model =  LSTMWithVariationalDropout( vocab_size, embed_dim, hidden_dim, output_dim)

train_lstm(model, train_data, word_dict, n_size, batch_size, epochs, device,dev_data)

Epoch 1/3: 100%|██████████| 9338/9338 [04:57<00:00, 31.35it/s]


Validation Loss after Epoch 1/3: 6.284131062809134
Epoch 1/3, Loss: 6.555533974450006


Epoch 2/3: 100%|██████████| 9338/9338 [03:59<00:00, 38.98it/s]


Validation Loss after Epoch 2/3: 6.107830102501047
Epoch 2/3, Loss: 5.967234891802423


Epoch 3/3: 100%|██████████| 9338/9338 [03:45<00:00, 41.41it/s]


Validation Loss after Epoch 3/3: 6.0784171897521135
Epoch 3/3, Loss: 5.777234085317494
Entraînement terminé!


In [21]:
def test_lstm(model, data_test, word_dict, n_size, batch_size, device):
    """
    Évalue le modèle sur les données de test.

    Args:
    - model: Le modèle MLP.
    - data_test: Données de test.
    - word_dict: Dictionnaire des mots.
    - ngram_size: Taille des n-grams.
    - batch_size: Taille du batch.
    - device: Le périphérique (CPU ou GPU).
    """
    
    # Tokenisation des phrases de test
    tokenized_test = tokenize_sentences(data_test['text'].tolist(), word_dict)
    perplexity = Perplexity()
    # Générer les n-grams pour le test
    input_sequences_test, target_words_test = [], []
    for sentence in tokenized_test:
        pairs = target_pairs([sentence],word_dict, n_size)
        for context, target in pairs:
            input_sequences_test.append(context)
            target_words_test.append(target)
    
    # Convertir les listes en tensors PyTorch
    input_tensor_test = torch.tensor(input_sequences_test, dtype=torch.long)
    target_tensor_test = torch.tensor(target_words_test, dtype=torch.long)
    
    # Créer un DataLoader pour itérer sur les batches
    test_dataset = TensorDataset(input_tensor_test, target_tensor_test)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Définir la fonction de perte
    criterion = nn.CrossEntropyLoss()
    
    # Évaluation
    model.eval()  # Passer en mode évaluation
    with torch.no_grad():
        test_loss = 0
        correct = 0
        total = 0
        for inputs, targets in  tqdm(test_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            # Calculer la perte
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            log_probs = torch.log_softmax(outputs, dim=1)  # Taille : [batch_size, vocab_size]
            log_probs_targets = log_probs[range(len(targets)), targets] 
            perplexity.add_sentence(log_probs_targets)
            # Calculer la précision
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        test_accuracy = correct / total
        print(f"Test Loss: {test_loss / len(test_dataloader)}, Test Accuracy: {test_accuracy * 100}%, Test Perplexity :{perplexity.compute_perplexity()}")


In [75]:
test_lstm(model, test_data, word_dict, n_size=5, batch_size=64, device=device)

100%|██████████| 1158/1158 [00:15<00:00, 75.51it/s]

Test Loss: 6.064233323247116, Test Accuracy: 14.889740613781749%, Test Perplexity :(430.11478070939734, 74098)
